In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import confusion_matrix
import os
import time
from os import listdir
from os.path import isfile, join

In [2]:
basePath = "/Users/macbookair/Desktop/università/computational intelligence/truth_predictor_project/models-data/data-from-eeglab/"
spectraBasePath = "/Users/macbookair/Desktop/università/computational intelligence/truth_predictor_project/spectra_epoch/"

eegDataTarget = pd.DataFrame()
eegDataNotTarget = pd.DataFrame()

spectraDataTarget = pd.DataFrame()
spectraDataNotTarget = pd.DataFrame()

allSubjectList = os.listdir(basePath)

for subjectName in allSubjectList:
    if os.path.isdir(basePath + subjectName):
        singleSubjectFiles = os.listdir(basePath + subjectName)
        for targetFileName in  singleSubjectFiles:
            if "epIs" in targetFileName and "EEG.csv" in targetFileName:
                #put file name in target array
                targetFile = pd.read_csv(basePath + subjectName + "/" + targetFileName,sep="\t")
                eegDataTarget = pd.concat([eegDataTarget,targetFile])
                spectraFileNameTarget = spectraBasePath + targetFileName.replace("EEG.csv","SPECTRA.csv")
                if os.path.exists(spectraFileNameTarget):
                    spectraTargetFile = pd.read_csv(spectraFileNameTarget, header=None)
                else:
                    spectraTargetFile = pd.read_csv(spectraBasePath + targetFileName.replace("EEG.csv","EEG-SPECTRA.csv"), header=None)
                spectraDataTarget = pd.concat([spectraDataTarget,spectraTargetFile])
            elif "epNot" in targetFileName and "EEG.csv" in targetFileName:
                #put file name in not_target array
                notTargetFile = pd.read_csv(basePath + subjectName + "/" + targetFileName,sep="\t")
                eegDataNotTarget = pd.concat([eegDataNotTarget,notTargetFile])
                spectraFileNameNotTarget = spectraBasePath + targetFileName.replace("EEG.csv","SPECTRA.csv")
                if os.path.exists(spectraFileNameNotTarget):
                    spectraNotTargetFile = pd.read_csv(spectraFileNameNotTarget, header=None)
                else:
                    spectraNotTargetFile = pd.read_csv(spectraBasePath + targetFileName.replace("EEG.csv","EEG-SPECTRA.csv"), header=None)
                spectraDataNotTarget = pd.concat([spectraDataTarget,spectraNotTargetFile])
                
eegDataTarget.rename(index=str, columns={" ": "Time"},inplace=True)
del eegDataTarget["Unnamed: 5"]

eegDataNotTarget.rename(index=str, columns={" ": "Time"},inplace=True)
del eegDataNotTarget["Unnamed: 5"]
print("")

In [64]:
windowStart = 0
windowEnd = 3000
numberOfSeconds = (windowEnd - windowStart)/ 1000
numberOfSamplesPerRound = int(numberOfSeconds * 256)

eegDataTargetReduced = eegDataTarget[(eegDataTarget["Time"] >= windowStart) & (eegDataTarget["Time"] < windowEnd)]
eegDataNotTargetReduced = eegDataNotTarget[(eegDataNotTarget["Time"] >= windowStart) & (eegDataNotTarget["Time"] < windowEnd)]

In [65]:
'''
del eegDataNotTargetReduced["Time"]
del eegDataTargetReduced["Time"]
'''

'\ndel eegDataNotTargetReduced["Time"]\ndel eegDataTargetReduced["Time"]\n'

In [66]:
eegDataTargetReducedNumpy = np.array(eegDataTargetReduced.values)
numberOfTargetValues = int(eegDataTargetReduced.shape[0]/numberOfSamplesPerRound)
trainingTargetSet = np.array(np.array_split(eegDataTargetReducedNumpy, eegDataTargetReduced.shape[0]/numberOfSamplesPerRound))

eegDataNotTargetReducedNumpy = np.array(eegDataNotTargetReduced.values)
trainingNotTargetSet = np.array(np.array_split(eegDataNotTargetReducedNumpy, eegDataNotTargetReduced.shape[0]/numberOfSamplesPerRound))

In [67]:
spectraDataTargetReducedNumpy = np.array(spectraDataTarget.values)
trainingSpectraTargetSet = np.array(np.array_split(spectraDataTargetReducedNumpy, spectraDataTarget.shape[0]/4))

spectraDataNotTargetReducedNumpy = np.array(spectraDataNotTarget.values)
trainingSpectraNotTargetSet = np.array(np.array_split(spectraDataNotTargetReducedNumpy, spectraDataNotTarget.shape[0]/4))


In [68]:
spectraDataTargetReducedNumpy.shape

(3220, 129)

In [69]:
trainingTargetSetReshaped = []
trainingNotTargetSetReshaped = []
for i in range(numberOfTargetValues - 1):
    trainingTargetSetReshaped.append(np.hstack((trainingTargetSet[i].flatten(),trainingSpectraTargetSet[i].flatten())))
    
for i in range(numberOfTargetValues - 1):
    trainingNotTargetSetReshaped.append(np.hstack((trainingNotTargetSet[i].flatten(),trainingSpectraNotTargetSet[i].flatten())))
    
trainingTargetSetReshaped = np.array(trainingTargetSetReshaped)
trainingNotTargetSetReshaped = np.array(trainingNotTargetSetReshaped)

print(trainingTargetSetReshaped.shape)
print(trainingNotTargetSetReshaped.shape)

(804, 4356)
(804, 4356)


In [70]:
#UNCOMMENT FOR TAKING AVERAGES
trainingTargetSetReshapedAndAveraged = np.zeros((trainingTargetSetReshaped.shape[0],trainingTargetSetReshaped.shape[1]))
trainingNotTargetSetReshapedAndAveraged = np.zeros((trainingTargetSetReshaped.shape[0],trainingTargetSetReshaped.shape[1]))

numberOfElectrodesLinesToBeAveragedPerSide = 15
numberOfElectrodesLinesToBeAveraged = (numberOfElectrodesLinesToBeAveragedPerSide * 2) + 1
numberOfAttributesConsidered = 5
totalNumberOfAttributesToJump = numberOfElectrodesLinesToBeAveragedPerSide * numberOfAttributesConsidered

for i in range(trainingTargetSetReshaped.shape[0]):
    for j in range(trainingTargetSetReshaped.shape[1]):
        if j < 3072 and j >= totalNumberOfAttributesToJump and j < trainingTargetSetReshaped.shape[1] - totalNumberOfAttributesToJump:
            z = 0
            while z <= numberOfElectrodesLinesToBeAveragedPerSide:
                if z != 0:
                    trainingTargetSetReshapedAndAveraged[i][j] += trainingTargetSetReshaped[i][j - (numberOfAttributesConsidered * z)]
                
                trainingTargetSetReshapedAndAveraged[i][j] += trainingTargetSetReshaped[i][j + (numberOfAttributesConsidered * z)]
                z = z + 1
            trainingTargetSetReshapedAndAveraged[i][j] = trainingTargetSetReshapedAndAveraged[i][j] / numberOfElectrodesLinesToBeAveraged
        else:
            trainingTargetSetReshapedAndAveraged[i][j] = trainingTargetSetReshaped[i][j]
    
    
for i in range(trainingTargetSetReshaped.shape[0]):
    for j in range(trainingTargetSetReshaped.shape[1]):
        if j < 3072 and j >= totalNumberOfAttributesToJump and j < trainingNotTargetSetReshaped.shape[1] - totalNumberOfAttributesToJump:
            z = 0
            while z <= numberOfElectrodesLinesToBeAveragedPerSide:
                if z != 0:
                    trainingNotTargetSetReshapedAndAveraged[i][j] += trainingNotTargetSetReshaped[i][j - (numberOfAttributesConsidered * z)]
                
                trainingNotTargetSetReshapedAndAveraged[i][j] += trainingNotTargetSetReshaped[i][j + (numberOfAttributesConsidered * z)]
                z = z + 1
            trainingNotTargetSetReshapedAndAveraged[i][j] = trainingNotTargetSetReshapedAndAveraged[i][j] / numberOfElectrodesLinesToBeAveraged
        else:
            trainingNotTargetSetReshapedAndAveraged[i][j] = trainingNotTargetSetReshaped[i][j]

print("")

(804, 4356)
(804, 4356)



In [105]:
print(trainingTargetSetReshapedAndAveraged.shape)
print(trainingNotTargetSetReshapedAndAveraged.shape)

(804, 4356)
(804, 4356)


In [106]:
trainingTargetSetReshaped = trainingTargetSetReshapedAndAveraged
trainingNotTargetSetReshaped = trainingNotTargetSetReshapedAndAveraged

validationTargetSetReshaped = trainingTargetSetReshaped[700:-1]
validationNotTargetSetReshaped = trainingNotTargetSetReshaped[700:-1]

In [107]:
numberOfTargetValues = 700
numberOfNotTarget = numberOfTargetValues

In [108]:
trainingTargetSetReshapedAndReduced = trainingTargetSetReshaped[0:numberOfTargetValues]
trainingNotTargetSetReshapedAndReduced = trainingNotTargetSetReshaped[0:numberOfTargetValues]

In [109]:
#HERE EQUIPROBABILITY
numberOfNotTarget = numberOfTargetValues
trainingNotTargetSet = trainingNotTargetSet[0:numberOfNotTarget]

trainingSpectraNotTargetSet = trainingSpectraNotTargetSet[0:numberOfNotTarget]

In [110]:
trainingSet = np.concatenate((trainingTargetSetReshapedAndReduced,trainingNotTargetSetReshapedAndReduced))

trainingSetLabelsOnes = np.ones((numberOfTargetValues,), dtype=int)
trainingSetLabelsZeros = np.zeros((numberOfNotTarget,), dtype=int)
trainingLabels = np.concatenate((trainingSetLabelsOnes,trainingSetLabelsZeros))
print(trainingSet.shape)
print(trainingLabels.shape)

(1400, 4356)
(1400,)


In [ ]:
rf300Score = 0
rf400Score = 0
rf450Score = 0

gradBoostScore = 0

rf300oobScore = 0
rf400oobScore = 0
rf450oobScore = 0

scaler = StandardScaler()
scaler.fit(trainingSet)
trainingSet = scaler.transform(trainingSet)

applyPCA = False
 
if applyPCA:
    #here fitting PCA
    pca = PCA(.999).fit(trainingSet)
    #transforming in lower dimensional space
    trainingSet = pca.transform(trainingSet)
    
start_time = time.time()

numberOfPermutations = 250

np.random.shuffle(trainingLabels)

scoreList = list()

for i in range(numberOfPermutations):
    
    train, test, labels, testLabels = train_test_split(trainingSet,trainingLabels,test_size=0.2,random_state=42)
    np.random.shuffle(trainingLabels)
    
    rf450 = RandomForestClassifier(n_estimators=450,n_jobs=-1)
    rf450.fit(train, labels)
    score = rf450.score(test,testLabels)
    rf450Score += score
    scoreList.append(score)
    
    '''
    rf300 = RandomForestClassifier(n_estimators=300,n_jobs=-1)
    rf300.fit(train, labels)
    rf300Score += rf300.score(test,testLabels)
    
    rf400 = RandomForestClassifier(n_estimators=400,n_jobs=-1)
    rf400.fit(train, labels)
    rf400Score += rf400.score(test,testLabels)
    
    gradBoost = GradientBoostingClassifier()
    gradBoost.fit(train,labels)
    gradBoostScore += gradBoost.score(test,testLabels)
    '''

'''
print("grad boost score:",end="")
print(gradBoostScore/numberOfFolds)
print("grad boost conf:")
print(confusion_matrix(testLabels, gradBoost.predict(test)))
print("")

print("RF-300 oob_score:",end="")
print(rf300oobScore)
print("")

print("RF-400 oob_score:",end="")
print(rf400oobScore)
print("")
'''

print("RF-450 average score:",end="")
print(rf450Score / numberOfPermutations)
print("")

end_time = time.time()

print("Execution time: ")
print(end_time - start_time)

In [ ]:
pd.DataFrame(scoreList).to_csv("/Users/macbookair/Desktop/università/computational intelligence/truth_predictor_project/permutation_scores/permutation_scores_1.csv")

In [113]:
validationSet = np.concatenate((validationTargetSetReshaped,validationNotTargetSetReshaped))

validationSet = scaler.transform(validationSet)

if applyPCA:
    validationSet = pca.transform(validationSet)

labelsNumber = int(validationSet.shape[0]/2)
validationSetLabelsOnes = np.ones((labelsNumber,), dtype=int)
validationSetLabelsZeros = np.zeros((labelsNumber,), dtype=int)
validationLabels = np.concatenate((validationSetLabelsOnes,validationSetLabelsZeros))
'''
print("RF-400 oob score:",end="")
print(rf400oob.score(validationSet,validationLabels))
print("RF conf:")
print(confusion_matrix(validationLabels, rf400oob.predict(validationSet)))
print("")

'''
print("RF-450:",end="")
print(rf450.score(validationSet,validationLabels))
print("RF conf:")
print(confusion_matrix(validationLabels, rf450.predict(validationSet)))
print("")
'''
print("grad boost score:",end="")
print(gradBoost.score(validationSet,validationLabels))
print("grad boost conf:")
print(confusion_matrix(validationLabels, gradBoost.predict(validationSet)))
print("")
'''
print("")

RF-450:0.5
RF conf:
[[41 62]
 [41 62]]


